In [1]:
pip install confluent_kafka

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 11.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [17]:
from confluent_kafka import Producer
from pyarrow import fs
import configparser, os, subprocess

In [98]:
from confluent_kafka import Producer
from pyarrow import fs
import configparser, os, subprocess

class Hdfs2Kafka(object):
    def __init__(self):
        classpath = subprocess.Popen(["/home/hadoop/hadoop/bin/hdfs", "classpath", "--glob"],
                         stdout=subprocess.PIPE).communicate()[0]
        os.environ['CLASSPATH'] = classpath.decode('utf-8')
        os.environ['ARROW_LIBHDFS_DIR'] = "/home/hadoop/hadoop/lib/native"
        self._hdfs = fs.HadoopFileSystem(host='192.168.0.160', port=8020, user='hadoop')

        kafka_brokers = "192.168.0.201:9092"
        kafka_resetType = 'earliest'

        conf = {'bootstrap.servers' : kafka_brokers, 'auto.offset.reset' : kafka_resetType}
        self._producer = Producer(conf)

    def getHdFileInfo(self, filename):
        f_Info = self._hdfs.get_file_info(filename)
        print(f'파일 종류 : {str(f_Info.type)}')
        print(f'파일 경로 : {str(f_Info.path)}')
        print(f'파일 크기 : {str(f_Info.size)}')
        print(f'파일 수정일자 : {str(f_Info.mtime)}')

    def readHdFile(self, filename):
        with self._hdfs.open_input_file(filename) as f:
            read_data = f.read().decode('utf-8').splitlines()
            #tmp = []
            #for line in read_data:
            #    tmp.append(line.split(","))
            #return tmp
            return [line.split(",") for line in read_data]

    def sendData2Kafka(self, topic, list_line):
        for data in list_line:
            str_tmp = ",".join(data).split(",")
            modified_data = ",".join(str_tmp[:2]) + "," + ",".join(str_tmp[4:])
            print(modified_data)
            self._producer.poll(0)
            self._producer.produce(topic, value=modified_data.encode('utf-8'), 
                                callback = kafka_producer_call)
            self._producer.flush()

def kafka_producer_call(err, msg):
    if err is not None:
        print (f"실패한 메시지 : error={err}, message={msg}")
    else:
        print(f"전달한 메시지: {msg.topic()} [{msg.partition()}] @ {msg.offset()}")
        print(f"massge.topic = {msg.topic()}")
        print(f"massge.timestamp = {msg.timestamp()}")
        print(f"massge.key = {msg.key()}")
        print(f"massge.value = {msg.value().decode('utf-8')}")
        print(f"massge.partition = {msg.partition()}")
        print(f"massge.offset = {msg.offset()}")


In [99]:
test01 = Hdfs2Kafka()

%4|1718334598.875|CONFWARN|rdkafka#producer-8| [thrd:app]: Configuration property auto.offset.reset is a consumer property and will be ignored by this producer instance


In [100]:
test01.getHdFileInfo("/user/rsh/civilian_force.csv")

파일 종류 : 2
파일 경로 : /user/rsh/civilian_force.csv
파일 크기 : 279777
파일 수정일자 : 2024-06-14 01:44:50+00:00


In [102]:
temp = test01.readHdFile("/user/rsh/civilian_force.csv")

In [104]:
test01.sendData2Kafka("test_0614", temp)

date,values,state,id,title,frequency_short,units_short,seasonal_adjustment_short
전달한 메시지: test_0614 [0] @ 2301
massge.topic = test_0614
massge.timestamp = (1, 1718335161331)
massge.key = None
massge.value = date,values,state,id,title,frequency_short,units_short,seasonal_adjustment_short
massge.partition = 0
massge.offset = 2301
1976-01-01,164014.0,Alaska,LAUST020000000000006A,Civilian Labor Force in Alaska,A,Persons,NSA
전달한 메시지: test_0614 [0] @ 2302
massge.topic = test_0614
massge.timestamp = (1, 1718335161338)
massge.key = None
massge.value = 1976-01-01,164014.0,Alaska,LAUST020000000000006A,Civilian Labor Force in Alaska,A,Persons,NSA
massge.partition = 0
massge.offset = 2302
1977-01-01,173460.0,Alaska,LAUST020000000000006A,Civilian Labor Force in Alaska,A,Persons,NSA
전달한 메시지: test_0614 [0] @ 2303
massge.topic = test_0614
massge.timestamp = (1, 1718335161343)
massge.key = None
massge.value = 1977-01-01,173460.0,Alaska,LAUST020000000000006A,Civilian Labor Force in Alaska,A,Persons,NSA


### 하둡 실습 2

In [105]:
from confluent_kafka import Consumer

In [108]:

conf = {'bootstrap.servers': "192.168.0.201:9092", 
        'group.id' : 'test-group',
         'auto.offset.reset' : 'earliest'}
consumer = Consumer(conf)


In [109]:
consumer.subscribe(['test_0614'])

In [111]:
while True:
    msg = consumer.poll(timeout=0.1)
    if msg is None:
        continue
    if msg.error():
        print(msg.error())
    else:
        print(msg.value().decode('utf-8'))
consumer.close()

%4|1718339326.453|SESSTMOUT|rdkafka#consumer-9| [thrd:main]: Consumer group session timed out (in join-state steady) after 216124 ms without a successful response from the group coordinator (broker 1, last error was Success): revoking assignment and rejoining group


KeyboardInterrupt: 

In [ ]:
# hdfs 저장하기 

from confluent_kafka import Producer
from pyarrow import fs
import configparser, os, subprocess

from pyarrow import fs 
import pandas as pd


classpath = subprocess.Popen(["/home/hadoop/hadoop/bin/hdfs", "classpath", "--glob"],
                 stdout=subprocess.PIPE).communicate()[0]
os.environ['CLASSPATH'] = classpath.decode('utf-8')
os.environ['ARROW_LIBHDFS_DIR'] = "/home/hadoop/hadoop/lib/native"
hdfs = fs.HadoopFileSystem(host='192.168.0.160', port=8020, user='hadoop')

df = pd.read_csv("/home/hadoop/class_data/civilian_force.csv")

with hdfs.open_output_stream("/encore/tmp2.csv") as fs:
    line = df.iloc[:100].to_csv()
    fs.write(line.encode())


# 실습

In [114]:
import requests
import xml.etree.ElementTree as ET
from confluent_kafka import Producer
import json
kafka_brokers = "192.168.0.201:9092"
kafka_resetType = "earliest"
conf = {'bootstrap.servers': kafka_brokers, 'auto.offset.reset': kafka_resetType}
producer =Producer(conf)
producer.poll(0)
topic = "RSH_KAFKA"


def fetch_json_data(url):
    com = ["curl", url]
    rt = subprocess.run(com,capture_output=True, text=True)
    json_data = json.loads(rt.stdout)
    return json_data

def kafka_producer_call(err, msg):
    if err is not None:
        print (f"실패한 메시지 : error={err}, message={msg}")
    else:
        print(f"전달한 메시지: {msg.topic()} [{msg.partition()}] @ {msg.offset()}")
       # print(f"massge.topic = {msg.topic()}")
       # print(f"massge.timestamp = {msg.timestamp()}")
      #  print(f"massge.key = {msg.key()}")
        #print(f"massge.value = {msg.value().decode('utf-8')}")
        #print(f"massge.partition = {msg.partition()}")
        print(f"massge.offset = {msg.offset()}")
    


def send_items_to_kafka(json_data):
    for item in items:
        item_json_str = json.dumps(item)
        producer.produce(topic, item_json_str.encode('utf-8'), callback=kafka_producer_call)
    
    producer.flush()


total_cnt = 1000

for pageNum in range(1, total_cnt):
    urls = f"https://apis.data.go.kr/B552584/EvCharger/getChargerStatus?serviceKey=34lIyWAciAsJlGtIZ4ltpLy2sLZDR%2BBRWvAv8RgoADNEd%2BKCgHe84XiSRUwL8JMMIubzsFW3ddjcNlhZHhvJIQ%3D%3D&pageNo={pageNum}&numOfRows=10&period=5&zcode=11&dataType=JSON"
    try:
        json_data = fetch_json_data(url)
        items = json_data.get("items",{}).get("item", [])
        send_items_to_kafka(items)
    except Exception as e:
        print(f"Erorr page{pageNum} {e}")


In [177]:
import json

In [226]:
kafka_brokers = "192.168.0.201:9092"
kafka_resetType = "earliest"
conf = {'bootstrap.servers': kafka_brokers, 'auto.offset.reset': kafka_resetType}
producer =Producer(conf)
producer.poll(0)
topic = "RSH_KAFKA"


def fetch_json_data(url):
    com = ["curl", url]
    rt = subprocess.run(com,capture_output=True, text=True)
    json_data = json.loads(rt.stdout)
    return json_data

def kafka_producer_call(err, msg):
    if err is not None:
        print (f"실패한 메시지 : error={err}, message={msg}")
    else:
        print(f"전달한 메시지: {msg.topic()} [{msg.partition()}] @ {msg.offset()}")
       # print(f"massge.topic = {msg.topic()}")
       # print(f"massge.timestamp = {msg.timestamp()}")
      #  print(f"massge.key = {msg.key()}")
        #print(f"massge.value = {msg.value().decode('utf-8')}")
        #print(f"massge.partition = {msg.partition()}")
        print(f"massge.offset = {msg.offset()}")
    


def send_items_to_kafka(json_data):
    for item in items:
        item_json_str = json.dumps(item)
        producer.produce(topic, item_json_str.encode('utf-8'), callback=kafka_producer_call)
    
    producer.flush()


total_cnt = 1000

for pageNum in range(1, total_cnt):
    urls = f"https://apis.data.go.kr/B552584/EvCharger/getChargerStatus?serviceKey=34lIyWAciAsJlGtIZ4ltpLy2sLZDR%2BBRWvAv8RgoADNEd%2BKCgHe84XiSRUwL8JMMIubzsFW3ddjcNlhZHhvJIQ%3D%3D&pageNo={pageNum}&numOfRows=10&period=5&zcode=11&dataType=JSON"
    try:
        json_data = fetch_json_data(url)
        items = json_data.get("items",{}).get("item", [])
        send_items_to_kafka(items)
    except Exception as e:
        print(f"Erorr page{pageNum} {e}")


%4|1718350410.458|CONFWARN|rdkafka#producer-25| [thrd:app]: Configuration property auto.offset.reset is a consumer property and will be ignored by this producer instance


전달한 메시지: RSH_KAFKA [0] @ 18350
massge.offset = 18350
전달한 메시지: RSH_KAFKA [0] @ 18351
massge.offset = 18351
전달한 메시지: RSH_KAFKA [0] @ 18352
massge.offset = 18352
전달한 메시지: RSH_KAFKA [0] @ 18353
massge.offset = 18353
전달한 메시지: RSH_KAFKA [0] @ 18354
massge.offset = 18354
전달한 메시지: RSH_KAFKA [0] @ 18355
massge.offset = 18355
전달한 메시지: RSH_KAFKA [0] @ 18356
massge.offset = 18356
전달한 메시지: RSH_KAFKA [0] @ 18357
massge.offset = 18357
전달한 메시지: RSH_KAFKA [0] @ 18358
massge.offset = 18358
전달한 메시지: RSH_KAFKA [0] @ 18359
massge.offset = 18359
Erorr page2 Expecting value: line 1 column 1 (char 0)
전달한 메시지: RSH_KAFKA [0] @ 18360
massge.offset = 18360
전달한 메시지: RSH_KAFKA [0] @ 18361
massge.offset = 18361
전달한 메시지: RSH_KAFKA [0] @ 18362
massge.offset = 18362
전달한 메시지: RSH_KAFKA [0] @ 18363
massge.offset = 18363
전달한 메시지: RSH_KAFKA [0] @ 18364
massge.offset = 18364
전달한 메시지: RSH_KAFKA [0] @ 18365
massge.offset = 18365
전달한 메시지: RSH_KAFKA [0] @ 18366
massge.offset = 18366
전달한 메시지: RSH_KAFKA [0] @ 18367
massge.offset 